In [21]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az
import pytensor


regions = pickle.load(open('test/luad34.regions.entropies.pkl', 'rb'))
data = regions[regions.chrom != 19].loc[:, 'log2_corrected'].values
batches = pm.Minibatch(data, batch_size=1_000)

states = np.array([[c, c-1] for c in range(1, 6)] + 
                    [[c, c] for c in range(6)] + 
                    [[c, c+1] for c in range(6)] +
                        [[c, 2] for c in range(6)])
n_states = states.shape[0]
K = 30
alpha = 1
with pm.Model() as model:

    w = pm.StickBreakingWeights("w", alpha, K-1)

    mu = pm.Normal("mu", mu=0, sigma=10, shape=K)
    sigma = pm.HalfNormal("sigma", sigma=10, shape=K)

    obs = pm.Mixture("obs", w, pm.StudentT.dist(mu=mu, sigma=sigma, nu=1, shape=K), observed=data)

    trace = pm.sample(1000, tune=1000, cores=1, chains=1, init='advi+adapt_diag')

az.plot_trace(trace)

# with pm.Model() as model:
#     w = pm.StickBreakingWeights('w', n_states)

#     n = pm.Beta('n', alpha=99, beta=1)
#     s = pm.Beta('s', alpha=10, beta=90)
#     phi = pm.Gamma('phi', alpha=3.5, beta=1)

#     w = pm.Dirichlet('w', a = np.ones(n_states))

#     mu = pm.Deterministic('mu', np.log((2 * n + (1 - n) * ((1 - s) * states[:, 0] + s * states[:, 1])) / (2 * n + (1 - n) * phi)))
#     lambda_ = pm.Gamma('lambda', alpha=3, beta=1, shape=n_states)
#     nu = pm.Gamma('nu', alpha=3, beta=1, shape=n_states)

#     obs = pm.Mixture('obs', w, pm.StudentT.dist(mu=mu, lam=lambda_, nu=nu, shape=n_states), observed=data)


Auto-assigning NUTS sampler...
Initializing NUTS using advi+adapt_diag...


Convergence achieved at 79200
Interrupted at 79,199 [39%]: Average Loss = -2.176e+05
Sequential sampling (1 chains in 1 job)
NUTS: [w, mu, sigma]
